In [ ]:
!pip install lyricsgenius

     |████████████████████████████████| 92kB 2.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
!pip install googletrans

     |████████████████████████████████| 61kB 2.7MB/s 
     |████████████████████████████████| 942kB 7.5MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 102kB 7.9MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=58b10b9f9825e8b96dd47f37a38eab09caecf0339f22739bd1a83f1e54ae0ea5
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=0cc5c89ae58f8bb49f88bc58760cb1c10b97bed044ed366f9cdaf12a4e38d0a8
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


In [ ]:
import numpy as np
import pandas as pd
import json
import time
import lyricsgenius
import  multiprocessing
import warnings
import random
import requests

from spotifyclient import SpotifyAPI
from googletrans import Translator


In [ ]:
warnings.filterwarnings('ignore')
random.seed(5)

In [ ]:
#Get the client id and client secret from SpotifyAPI
client_id = '116037a141a04c628cdadaf11310f7d8'
client_secret = 'f505511e896d4b8fb4cd27c4e771f9f6'

genius = lyricsgenius.Genius('p2U70gp0xq5TWV10R8iJXI5uz4DjNseRup1FA63S8-3ZgQBOMyCneDzCew89BXVb')

spotify = SpotifyAPI(client_id, client_secret)

In [ ]:
genius.verbose = False # Turn off status messages
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.excluded_terms = ["(Remix)", "(Live)"]

In [ ]:
def preprocess_df(df,query):

  """ This function is used to preprocess the given 
      dataframe

      Arguments :
        1) df(DataFrame) : The dataframe to be preprocessed
        2) query(str)    : The type of genre

      Returns : 
        The preprocessed dataframe

  """
  df.drop_duplicates(keep='first',inplace=True)
  df.reset_index(drop=True,inplace=True)
  df.rename(columns={'artist':'Artist','song':'Song'},inplace=True)
  print('Number of songs returned by the Spotify API of genre {}\
  is {}'.format(query.upper(),len(df)))

  return df

In [ ]:
def get_song_name(query,search_type):

  """ This function is used to return the song titles 
      and also the artist names correspoding to each 
      query and type.

      Arguments:

        1) query(str) : The type of genre.
        2) search_type(str)  : The form in which the 
                               songs are returned.

      Returns:

        A list of dictionaries containing the song name
        and it's corresponding artists.

  """
  play_data = spotify.search(query = query, search_type = search_type)
  list_of_songs = []

  for i in range(len(play_data['playlists']['items'])):
    track_data = spotify.get_playlist_tracks(play_data['playlists']['items']
                                             [i]['id'])

    for j in range(len(track_data['items'])-1):     
      songs_and_artists={}
      try:
        songs_and_artists['artist'] = track_data['items'][j+1]['track'] \
                                                ['artists'][0]['name']
      except:
        songs_and_artists['artist'] = 'None' 
      try:
        songs_and_artists['song']=track_data['items'][j+1]['track']['name']
      except:
        songs_and_artists['song'] = 'None'

      list_of_songs.append(songs_and_artists)      

  final_list = [i for i in list_of_songs if not (i['artist'] == 'None' or 
                                                 i['song'] == 'None')]
  df_final = pd.DataFrame(final_list)
  
  return preprocess_df(df_final,query) 

In [ ]:
#def genius_api(artist_name,song_name):

  """ This function is used to return the lyrics of a song 
      using the GeniusAPI.

      Arguments :

        1) artist_name(str) : Name of the artist.
        2) song_name(str)   : Name of the song.

     Returns :

       The lyrics of the song

  """
  #try:
   # song = genius.search_song(song_name,artist_name,get_full_info=True)
    #lyrics = song.lyrics
  #except:
   # lyrics = np.nan

  #return lyrics

In [ ]:
def genius_api(artist_name,song_name):

  """ This function is used to return the lyrics of a song 
      using the GeniusAPI.

      Arguments :

        1) artist_name(str) : Name of the artist.
        2) song_name(str)   : Name of the song.

      Returns :

        The lyrics of the song

  """
  try:
    song = genius.search_song(song_name,
                            artist_name,
                            get_full_info=False)
    lyrics = song.lyrics
    
  except requests.exceptions.Timeout:
    return np.nan

  except AttributeError:
    i = 0
    while song is None:
      if i == 5:
        return np.nan
        break
      song = genius.search_song(song_name,
                                artist_name,
                                get_full_info=False)
      i+=1
      if song is not None:
        lyrics= song.lyrics
        break

  return lyrics

In [ ]:
def get_song_lyrics(df,p=50):

  """ This function is used to get the song lyrics using 
      multiprocessing.

      Arguments:
        
        1) df(DataFrame) : The dataset to do multiprocessing on.
        2) p(int)        : The number of processes.

      Returns :

        The dataframe which contains the lyrics.

  """
  #with multiprocessing.dummy.Pool(processes=p) as pool:
  pool = multiprocessing.Pool(50)
  df['Lyrics'] = pool.starmap(genius_api,
                              zip(df['Artist'],df['Song']))
  pool.terminate()
  pool.join()

  return df

In [ ]:
#Getting the song names of genre 'Love'
df_title_love = get_song_name('love','playlist')

q=love&type=playlist&limit=50
Number of songs returned by the Spotify API of genre LOVE  is 3021


In [ ]:
df_title_love.tail()

,Artist,Song
3016,Skylar Belice,Bad for You
3017,The Script,Breakeven
3018,Josephina,Dope Girl
3019,Sam Smith,Too Good At Goodbyes
3020,Ananya Birla,Let There Be Love


In [ ]:
df_title_love.isnull().sum()

Artist    0
Song      0
dtype: int64

In [ ]:
#Getting the song lyrics corresponding to the song name and artist name
df_love = get_song_lyrics(df_title_love)

In [ ]:
df_love.isnull().sum()

Artist      0
Song        0
Lyrics    527
dtype: int64

In [ ]:
df_love.dropna(axis=0,inplace=True)

In [ ]:
df_love.head()

,Artist,Song,Lyrics
1,Dan + Shay,I Should Probably Go To Bed,All of my friends finally convinced me to get ...
2,Lost Frequencies,Don't Leave Me Now,Scene 08: Timber & The Owls\nScene 09: The TV ...
4,Joel Corry,Head & Heart (feat. MNEK) - Acoustic,"Oh my god, oh my god, this feeling's just begu..."
5,MisterWives,3 small words,Your watch broke\nAnd I hope that it stopped t...
6,Emily Burns,"I Love You, You're The Worst",Do you string me along for the thrill?\nKnock ...


In [ ]:
df_love.to_csv('love_genre.csv')

In [ ]:
#Getting the song names of genre 'Sad'
df_title_sad = get_song_name('sad','playlist')

q=sad&type=playlist&limit=50
Number of songs returned by the Spotify API of genre SAD  is 2855


In [ ]:
df_title_sad.head(34)

,Artist,Song
0,NF,Let You Down
1,Russ,Losin Control
2,Drake,Doing It Wrong
3,Khalid,Coaster
4,Logic,1-800-273-8255
5,James Arthur,Say You Won't Let Go
6,The Killers,Just Another Girl
7,Khalid,Reasons
8,The Weeknd,Die For You
9,G-Eazy,Think About You (feat. Quiñ)


In [ ]:
#Getting the song lyrics corresponding to the song name and artist name
df_sad = get_song_lyrics(df_title_sad)

In [ ]:
df_sad.head(24)

,Artist,Song,Lyrics
0,NF,Let You Down,Feels like we're on the edge right now\nI wish...
1,Russ,Losin Control,"She's falling in love now, losing control now\..."
2,Drake,Doing It Wrong,When a good thing goes bad it's not the end of...
3,Khalid,Coaster,"As time passes, I feel so low\nSearchin' for p..."
4,Logic,1-800-273-8255,"I've been on the low, I been taking my time\nI..."
5,James Arthur,Say You Won't Let Go,I met you in the dark\nYou lit me up\nYou made...
6,The Killers,Just Another Girl,Step out into the Indian dust\nI can feel the ...
7,Khalid,Reasons,Evade my intellect\nFeed into my introspect\nW...
8,The Weeknd,Die For You,I'm findin' ways to articulate the feeling I'm...
9,G-Eazy,Think About You (feat. Quiñ),'Cause I do\n'Cause I do\n'Cause I do\nKeep wr...


In [ ]:
df_sad.isnull().sum()

Artist      0
Song        0
Lyrics    555
dtype: int64

In [ ]:
df_sad.to_csv('sad_genre.csv')

In [ ]:
#Getting the song name of genre 'Motivation'
df_title_motivation = get_song_name('motivation','playlist')

q=motivation&type=playlist&limit=50
Number of songs returned by the Spotify API of genre MOTIVATION  is 2831


In [ ]:
df_title_motivation.head()

,Artist,Song
0,Lost Frequencies,Don't Leave Me Now
1,BUNT.,Unbreakable (feat. Clarence Coffee Jr.)
2,Dua Lipa,Hallucinate - Paul Woolford Remix
3,Sam Feldt,You Should Know (feat. Craig Smart)
4,Bob Marley & The Wailers,Sun Is Shining


In [ ]:
#Getting song lyrics corresponding to the song name and artist
df_title = get_song_lyrics(df_title_motivation)

In [ ]:
df_title.head()

,Artist,Song,Lyrics
0,Lost Frequencies,Don't Leave Me Now,Scene 08: Timber & The Owls\nScene 09: The TV ...
1,BUNT.,Unbreakable (feat. Clarence Coffee Jr.),"Renegade by nature, child of the wild\nWrestli..."
2,Dua Lipa,Hallucinate - Paul Woolford Remix,NaN
3,Sam Feldt,You Should Know (feat. Craig Smart),I can't shut off this silence\nWhen I feel he'...
4,Bob Marley & The Wailers,Sun Is Shining,"Sun is shining, the weather is sweet\nMake you..."


In [ ]:
df_title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2831 entries, 0 to 2830
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Artist  2831 non-null   object
 1   Song    2831 non-null   object
 2   Lyrics  2220 non-null   object
dtypes: object(3)
memory usage: 66.5+ KB


In [ ]:
df_title.isnull().sum()

Artist      0
Song        0
Lyrics    611
dtype: int64

In [ ]:
df_title.dropna(axis=0,inplace=True)

In [ ]:
df_title.to_csv('motivation_genre.csv')

In [ ]:
import json

with open('workout.json', 'w') as json_file:
  json.dump(workout, json_file)

In [ ]:
df_

In [ ]:
import lyricsgenius
genius = lyricsgenius.Genius('p2U70gp0xq5TWV10R8iJXI5uz4DjNseRup1FA63S8-3ZgQBOMyCneDzCew89BXVb')

In [ ]:
pop = pd.DataFrame({'Artist':['Harry Styles','Bruno Mars','ffgf'],'Song':['Sign of the Times','Talking to the Moon','fdsd']})